## 2nd step arrange data to get more accuracy

In [1]:
import pandas as pd
import json

In [2]:
# Read the data from the CSV file
# dt_arr = data_arrange
dt_arr = pd.read_csv('C:/Users/irfanizam/workspace/FYP-Example/FinalFinalFYP/Data2/CleanedLt.csv')

In [3]:
x = dt_arr['tag']
y = dt_arr['summary']

In [4]:
# dt_lt_arr = data long text arrange
# pre_arr_dt_lt = pre arrange data long text
selected_columns = ['tag', 'summary']  
dt_lt_arr = dt_arr[selected_columns]
dt_lt_arr.to_csv('C:/Users/irfanizam/workspace/FYP-Example/FinalFinalFyp/Data2/pre_arr_dt_lt.csv', index=False) 

In [5]:
multiclass_lt = pd.read_csv('C:/Users/irfanizam/workspace/FYP-Example/FinalFinalFyp/Data2/pre_arr_dt_lt.csv')

In [6]:
multiclass_lt = pd.get_dummies(multiclass_lt, columns=['tag'], prefix='', prefix_sep='')

In [7]:
print(multiclass_lt.columns)

Index(['summary', '1', '2', '3', '4', '5', '6', '7'], dtype='object')


In [8]:
multiclass_lt = multiclass_lt.groupby('summary').max().reset_index()

In [9]:
multiclass_lt.to_csv('C:/Users/irfanizam/workspace/FYP-Example/FinalFinalFyp/Data2/ArrangedLt.csv', index=False)  